In [21]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
print("TensorFlow version: ",tf.version.VERSION)

import pandas as pd
import numpy as np
import urllib
import json
import datetime

from utils import get_daily_demand_data

TensorFlow version:  2.3.4


In [1]:
!pip install --user google-cloud-bigquery==1.25.0

     |████████████████████████████████| 169 kB 7.4 MB/s eta 0:00:01     |███████████▋                    | 61 kB 7.4 MB/s eta 0:00:01
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 1.28.0
    Uninstalling google-cloud-bigquery-1.28.0:
      Successfully uninstalled google-cloud-bigquery-1.28.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.42.0 requires google-resumable-media<3.0dev,>=1.3.0; python_version >= "3.6", but you have google-resumable-media 0.5.1 which is incompatible.


In [ ]:
# Change with your own bucket and project below:
BUCKET =  "<BUCKET>"
PROJECT = "<PROJECT>"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT

In [13]:
start_date = '2019-04-01'
end_date = '2020-03-31' 

NORM_VALUE = 1e6 # divided all values normalization

values, times = get_daily_demand_data(start_date, end_date, write_json=True)
values = values / NORM_VALUE

In [20]:
!gsutil cp -r data gs://qwiklabs-gcp-01-2c981bf9193f

Copying file://data/data.json [Content-Type=application/json]...
/ [1 files][ 30.8 KiB/ 30.8 KiB]                                                
Operation completed over 1 objects/30.8 KiB.                                     


## Create BQ Dataset

In [14]:
# https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/feature_engineering/labs/1_bqml_basic_feat_eng_bqml-lab.ipynb

In [4]:
%%bash
# Create a BigQuery dataset for feat_eng if it doesn't exist

datasetexists=$(bq ls -d | grep -w feat_eng)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset"
    
    bq --location=US mk --dataset \
        --description 'Demand prediction' \
        $PROJECT:demand_pred
   echo "\nHere are your current datasets:"
   bq ls
fi

Creating BigQuery dataset titled: feat_eng
Dataset 'qwiklabs-gcp-01-2c981bf9193f:demand_pred' successfully created.
\nHere are your current datasets:
   datasetId   
 ------------- 
  demand_pred  


In [ ]:
%%bigquery
 
CREATE OR REPLACE TABLE
  demand_pred.training_data AS
SELECT *
FROM
  `nyc-tlc.yellow.trips`
WHERE
  MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 10000) = 1
  AND fare_amount >= 2.5
  AND passenger_count > 0
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45

## Create Model!

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL
  feat_eng.baseline_model OPTIONS (model_type='linear_reg',
    input_label_cols=['fare_amount']) AS
SELECT
  fare_amount,
  passengers,
  pickup_datetime,
  pickuplon,
  pickuplat,
  dropofflon,
  dropofflat
FROM
  feat_eng.feateng_training_data